In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import shutil
%matplotlib inline 

In [36]:
df = pd.read_csv("/content/drive/MyDrive/Hackathon_Models/Age & Gender Model/age_gender.csv")

In [37]:
df.sample(10)

,age,ethnicity,gender,img_name,pixels
11933,32,1,1,20170113000731466.jpg.chip.jpg,140 127 127 94 80 60 35 32 24 25 67 123 159 17...
15149,39,0,0,20170104174316011.jpg.chip.jpg,53 57 59 77 98 122 124 131 148 150 148 146 149...
16940,45,0,0,20170117203225383.jpg.chip.jpg,87 105 118 132 150 154 157 158 162 165 169 171...
19984,57,0,0,20170117191706202.jpg.chip.jpg,96 105 126 148 166 170 166 164 169 168 167 174...
15698,40,4,1,20170109003506229.jpg.chip.jpg,28 17 24 22 21 21 23 23 24 25 28 25 15 10 7 17...
22260,73,0,0,20170104213653518.jpg.chip.jpg,84 143 124 155 162 170 166 181 186 192 204 204...
15952,40,0,0,20170117152243349.jpg.chip.jpg,196 196 198 198 198 200 158 41 41 80 98 113 11...
9641,28,0,1,20170116221705370.jpg.chip.jpg,127 115 124 114 99 95 104 128 161 185 201 211 ...
12804,34,0,0,20170104170338177.jpg.chip.jpg,108 98 90 73 109 168 190 191 198 196 190 182 1...
20073,58,0,0,20170105173557845.jpg.chip.jpg,30 30 29 28 27 29 26 22 22 22 16 13 13 14 15 1...


In [38]:
for i in range(0,df.shape[0]):    
  df['pixels'][i] = np.array(df['pixels'][i].split(' ')).reshape(48,48)

<ipython-input-38-dcb21e7fedf0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pixels'][i] = np.array(df['pixels'][i].split(' ')).reshape(48,48)


In [39]:
df.isnull().sum()

age          0
ethnicity    0
gender       0
img_name     0
pixels       0
dtype: int64

In [40]:
df['age'].nunique()

104

In [41]:
df['ethnicity'].nunique()

5

In [42]:
df["age."] = pd.cut(df["age"],bins=[0., 10., 20.0, 30., 50.,70.,np.inf],labels=[0,1,2,3,4,5])

In [43]:
df.columns

Index(['age', 'ethnicity', 'gender', 'img_name', 'pixels', 'age.'], dtype='object')

In [44]:
df = df.drop(['img_name','age'],axis = 1)

In [45]:
df.head(2)

,ethnicity,gender,pixels,age.
0,2,0,"[[129, 128, 128, 126, 127, 130, 133, 135, 139,...",0
1,2,0,"[[164, 74, 111, 168, 169, 171, 175, 182, 184, ...",0


In [46]:
df['ethnicity'] = df['ethnicity'].astype(np.float32)
df['gender'] = df['gender'].astype(np.float32)
df['age.'] = df['age.'].astype(np.float32)

In [66]:
for i in range(0,df.shape[0]):
  df['pixels'][i] = df['pixels'][i].astype(np.float32).reshape(48,48,1)

<ipython-input-66-d87f2df839d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pixels'][i] = df['pixels'][i].astype(np.float32).reshape(48,48,1)


In [68]:
df['pixels'] = df['pixels'].apply(lambda x: x/255)


In [69]:
X = np.array(df['pixels'].tolist())

In [70]:
X = X.reshape(X.shape[0],48,48,1)

In [72]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,np.array(df['age.']),test_size = 0.2, random_state = 30)

In [73]:
model = Sequential()

In [74]:
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

In [75]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

In [76]:
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

In [77]:
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

In [78]:
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

In [79]:
model.add(Dense(6, activation='softmax'))

In [80]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [57]:
#X_train = np.array([np.array(val) for val in X_train])

In [81]:
y_train

array([0., 4., 5., ..., 3., 2., 2.], dtype=float32)

In [82]:
from keras.utils import to_categorical

In [83]:
one_hot_label = to_categorical(y_train,num_classes = 6)

In [84]:
y_train

array([0., 4., 5., ..., 3., 2., 2.], dtype=float32)

In [62]:
one_hot_label

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [85]:
#X_test = np.array([np.array(val) for val in X_test])
one_hot_label2 = to_categorical(y_test,num_classes = 6)

In [86]:
X_train.shape

(18964, 48, 48, 1)

In [87]:
history=model.fit(X_train,one_hot_label,steps_per_epoch=100, epochs=150, validation_data=(X_test,one_hot_label2), validation_steps=100)

Epoch 1/150
100/100 [==============================] - ETA: 0s - loss: 1.5444 - accuracy: 0.3632

100/100 [==============================] - 16s 60ms/step - loss: 1.5444 - accuracy: 0.3632 - val_loss: 1.3101 - val_accuracy: 0.4740
Epoch 2/150
100/100 [==============================] - 4s 39ms/step - loss: 1.1852 - accuracy: 0.5195
Epoch 3/150
100/100 [==============================] - 4s 40ms/step - loss: 1.0481 - accuracy: 0.5669
Epoch 4/150
100/100 [==============================] - 4s 40ms/step - loss: 0.9712 - accuracy: 0.5945
Epoch 5/150
100/100 [==============================] - 4s 40ms/step - loss: 0.9049 - accuracy: 0.6240
Epoch 6/150
100/100 [==============================] - 4s 40ms/step - loss: 0.8585 - accuracy: 0.6432
Epoch 7/150
100/100 [==============================] - 4s 40ms/step - loss: 0.8179 - accuracy: 0.6591
Epoch 8/150
100/100 [==============================] - 4s 40ms/step - loss: 0.7861 - accuracy: 0.6690
Epoch 9/150
100/100 [==============================] - 4s 40ms/step - loss: 0.7541 - accuracy: 0.6843
Epoch 10/150
100/100 [=============================

In [88]:
model.save('age_model.h5')

In [98]:
import cv2
import numpy as np
from keras.models import load_model

model=load_model('age_model.h5')

faceDetect=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

frame=cv2.imread("/content/photo-1630305131239-c8df91783f10.jpg") #baby pic 
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces= faceDetect.detectMultiScale(gray, 1.3, 3)
x,y,w,h = faces[0]
sub_face_img=gray[y:y+h, x:x+w]
resized=cv2.resize(sub_face_img,(48,48))
normalize=resized/255.0
reshaped=np.reshape(normalize, (1, 48, 48, 1))
result=model.predict(reshaped)
label=np.argmax(result, axis=1)[0]
print(label)

1/1 [==============================] - 0s 118ms/step
0


GENDER

In [115]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,np.array(df['gender']),test_size = 0.2, random_state = 30)

In [116]:
model2 = Sequential()

In [117]:
model2.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

In [118]:
model2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.1))

In [119]:
model2.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.1))

In [120]:
model2.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.1))

In [121]:
model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.2))

In [122]:
model2.add(Dense(2, activation='sigmoid'))

In [123]:
model2.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [124]:
one_hot_label_gender = to_categorical(y_train,num_classes = 2)

In [125]:
one_hot_label_gender2 = to_categorical(y_test,num_classes = 2)

In [126]:
history=model2.fit(X_train,one_hot_label_gender,steps_per_epoch=100, epochs=40, validation_data=(X_test,one_hot_label_gender2), validation_steps=100)

Epoch 1/40
100/100 [==============================] - ETA: 0s - loss: 0.5382 - accuracy: 0.7183

100/100 [==============================] - 5s 48ms/step - loss: 0.5382 - accuracy: 0.7183 - val_loss: 0.3736 - val_accuracy: 0.8386
Epoch 2/40
100/100 [==============================] - 4s 40ms/step - loss: 0.3510 - accuracy: 0.8439
Epoch 3/40
100/100 [==============================] - 4s 40ms/step - loss: 0.2976 - accuracy: 0.8726
Epoch 4/40
100/100 [==============================] - 4s 40ms/step - loss: 0.2637 - accuracy: 0.8866
Epoch 5/40
100/100 [==============================] - 4s 40ms/step - loss: 0.2428 - accuracy: 0.8946
Epoch 6/40
100/100 [==============================] - 4s 40ms/step - loss: 0.2195 - accuracy: 0.9078
Epoch 7/40
100/100 [==============================] - 4s 40ms/step - loss: 0.2083 - accuracy: 0.9125
Epoch 8/40
100/100 [==============================] - 4s 40ms/step - loss: 0.1899 - accuracy: 0.9226
Epoch 9/40
100/100 [==============================] - 4s 40ms/step - loss: 0.1747 - accuracy: 0.9299
Epoch 10/40
100/100 [==============================] - 4s 40

In [127]:
model2.save('gender_model.h5')

In [133]:
# 0 : Male & 1 : Female
model2=load_model('gender_model.h5')

faceDetect=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

frame=cv2.imread("/content/download (3).jpg")  
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces= faceDetect.detectMultiScale(gray, 1.3, 3)
x,y,w,h = faces[0]
sub_face_img=gray[y:y+h, x:x+w]
resized=cv2.resize(sub_face_img,(48,48))
normalize=resized/255.0
reshaped=np.reshape(normalize, (1, 48, 48, 1))
result=model2.predict(reshaped)
label=np.argmax(result, axis=1)[0]
print(label)

1/1 [==============================] - 0s 74ms/step
0
